[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FeetCodingHommy/kakao-arena-melon-playlist-dataset-and-more/blob/master/colab_notebooks/crawl_test.ipynb)

# 세팅

In [ ]:
!git clone https://github.com/FeetCodingHommy/kakao-arena-melon-playlist-dataset-and-more.git

In [ ]:
!pip install -qq scrapy
!pip install -qq selenium

In [ ]:
import sys
sys.path.append("/content/kakao-arena-melon-playlist-dataset-and-more/melon_scrapy") # custom_settings에 필요
import melon_scrapy

In [ ]:
# 코랩 세션 끊김 방지
#   아래 코드를 크롬 F12 콘솔에 실행

%%javascript
// function ClickConnect(){
//     console.log("9분!"); 
//     document.querySelector("colab-toolbar-button").click() 
// }
// setInterval(ClickConnect, 9 * 60 * 1000)
console.log("Melon Crawler Test")

  * 서버에 바로 다운 받을 수 있는 API는 제공하지 않음 ([관련](https://arena.kakao.com/forum/topics/193))
    * 직접 다운해서 구글 드라이브에 올리고 마운트 시키는 방법
    * 임시로 구글 드라이브에 있는 파일을 공유해서 다운받는 방법

In [ ]:
# 임시로 구글 드라이브에 있는 파일을 공유해서 다운받는 방법 - 나중에 링크 비활성화 필수

!gdown --id 1JKxtYManLah1kZ6C0KqlTeqn0uKUQAc7
!mv /content/song_meta.json /content/kakao-arena-melon-playlist-dataset-and-more/_scrap_input/